In [2]:
# ======================= [ 1. 환경 및 패키지 준비 ] =======================
!pip install -q timm pandas openpyxl

import os
import random
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score

# ======================= [ 2. 구글 드라이브 마운트 및 경로 지정 ] =======================
from google.colab import drive
drive.mount('/content/drive')

# -- 여기를 본인 데이터에 맞게 변경하세요 --
project_dir = "/content/drive/MyDrive"        # 예시: 드라이브에 'project' 폴더가 있을 때
img_dir     = f"{project_dir}/dataset"                # 이미지 폴더 (001.JPG ~ 101.JPG)
weight_path = f"{project_dir}/weight.xlsx"            # 무게 정보 엑셀 파일
csv_path    = f"{project_dir}/train_3class.csv"       # 자동 생성/로드되는 csv

# ======================= [ 3. CSV 자동 생성 ] =======================
if not os.path.exists(csv_path):
    df_w = pd.read_excel(weight_path)
    df_w["filename"] = df_w["No"].astype(str).str.zfill(3) + ".JPG"
    def classify(kg):
        if kg <= 5000: return "light"
        if kg <= 10000: return "medium"
        return "heavy"
    df_w["weight_class"] = df_w["KG"].apply(classify)
    df_w[["filename", "weight_class"]].to_csv(csv_path, index=False)
    print(f"CSV saved → {csv_path}")

df = pd.read_csv(csv_path)

# ======================= [ 4. Dataset 정의 (on-the-fly 증강)] =======================
class AugScrapDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform, label_encoder, repeat=10, augment=True):
        self.df = dataframe.reset_index(drop=True).copy()
        self.img_dir = img_dir
        self.transform = transform
        self.le = label_encoder
        self.df["class_idx"] = self.le.fit_transform(self.df["weight_class"])
        self.repeat = repeat
        self.augment = augment
        self.aug_ops = [
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.RandomVerticalFlip(p=1.0),
            transforms.ColorJitter(0.4,0.4,0.4)
        ]

    def __len__(self):
        return len(self.df) * self.repeat if self.augment else len(self.df)

    def __getitem__(self, idx):
        base_idx = idx % len(self.df)
        row = self.df.iloc[base_idx]
        img_path = os.path.join(self.img_dir, row["filename"])
        img = Image.open(img_path).convert("RGB")
        y = torch.tensor(row["class_idx"])
        tr = self.transform
        if self.augment and idx // len(self.df) > 0:
            op = random.choice(self.aug_ops)
            tr = transforms.Compose([
                transforms.Resize((224,224)),
                op,
                transforms.ToTensor(),
                transforms.Normalize([0.5]*3, [0.5]*3)
            ])
        return tr(img), y

# Validation용 Dataset (증강 X)
class ScrapDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform, label_encoder):
        self.df = dataframe.reset_index(drop=True).copy()
        self.img_dir = img_dir
        self.transform = transform
        self.le = label_encoder
        self.df["class_idx"] = self.le.fit_transform(self.df["weight_class"])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["filename"])
        img = Image.open(img_path).convert("RGB")
        y = torch.tensor(row["class_idx"])
        return self.transform(img), y

# ======================= [ 5. Transform 및 K-Fold ] =======================
base_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_idx, val_idx = list(kf.split(df))[0]

train_df = df.iloc[train_idx].copy()
val_df   = df.iloc[val_idx]
label_encoder = LabelEncoder()

# ======================= [ 6. DataLoader 준비 ] =======================
batch_size = 8
repeat = 10     # 증강 배수

train_ds = AugScrapDataset(train_df, img_dir, base_transform, label_encoder, repeat=repeat, augment=True)
val_ds   = ScrapDataset(val_df, img_dir, base_transform, label_encoder)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# ======================= [ 7. RegNetY_002 모델 정의 ] =======================
class RegNetY002Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model(
            "regnety_002", pretrained=True, num_classes=3
        )
    def forward(self, x):
        return self.backbone(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = RegNetY002Classifier().to(device)

# ======================= [ 8. 학습/평가 함수 ] =======================
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

from torch.amp import autocast, GradScaler
scaler = GradScaler()
EPOCHS = 10

def evaluate():
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for x, y in val_loader:
            o = model(x.to(device))
            preds.extend(o.argmax(1).cpu())
            labels.extend(y)
    acc = (torch.tensor(preds) == torch.tensor(labels)).float().mean()*100
    pr  = precision_score(labels, preds, average="macro", zero_division=0)
    rc  = recall_score(labels,  preds, average="macro", zero_division=0)
    f1  = f1_score(labels,      preds, average="macro", zero_division=0)
    return acc.item(), pr, rc, f1

ACCUM = 2  # gradient accumulation (배치 부족시 늘리세요)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    for step, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        with autocast(device_type="cuda"):
            loss = criterion(model(x), y) / ACCUM
        scaler.scale(loss).backward()
        if (step + 1) % ACCUM == 0 or (step + 1) == len(train_loader):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        total_loss += loss.item()
    acc, pr, rc, f1 = evaluate()
    print(f"Epoch {epoch+1:02d} | loss {total_loss/len(train_loader):.4f} | acc {acc:.2f}% | f1 {f1:.4f}")

print("\n=== Final metrics ===")
print("Accuracy :", acc, "%")
print("Precision:", pr)
print("Recall   :", rc)
print("F1 Score :", f1)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

Epoch 01 | loss 0.4212 | acc 66.67% | f1 0.6723
Epoch 02 | loss 0.2094 | acc 76.19% | f1 0.7628
Epoch 03 | loss 0.0912 | acc 57.14% | f1 0.5739
Epoch 04 | loss 0.0530 | acc 80.95% | f1 0.8056
Epoch 05 | loss 0.0440 | acc 76.19% | f1 0.7643
Epoch 06 | loss 0.0487 | acc 66.67% | f1 0.6747
Epoch 07 | loss 0.0283 | acc 71.43% | f1 0.7212
Epoch 08 | loss 0.0316 | acc 66.67% | f1 0.6747
Epoch 09 | loss 0.0187 | acc 66.67% | f1 0.6747
Epoch 10 | loss 0.0261 | acc 66.67% | f1 0.6691

=== Final metrics ===
Accuracy : 66.66667175292969 %
Precision: 0.6765873015873015
Recall   : 0.6666666666666666
F1 Score : 0.6691086691086691
